In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
import random
import urllib.parse
from bs4 import BeautifulSoup
import json

In [2]:
with open('links.txt',encoding='utf-8') as f:
    links_raw = f.readlines()
print(len(links_raw))
links = list(set(links_raw))
print(len((links)))

187
143


In [3]:
def scroll_to_bottom(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")
    count = 0
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait for page to load
        time.sleep(random.uniform(1.5, 2.5))

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            count += 1
            if count == 50:
                break
        else:
            last_height = new_height
            count = 0

        # Simulate human-like scrolling by scrolling in smaller increments
        #for i in range(3):
         #   driver.find_element_by_tag_name('body').send_keys(Keys.PAGE_DOWN)
          #  time.sleep(random.uniform(0.5, 1))
def extract_text(url,file_name):
    print('starting******************wait until closing**********')
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(options=options)    
    driver.get(url)
    scroll_to_bottom(driver)
    print('scrolling completed')
    # Find all clickable "மேலும் காண்க" elements
    wait = WebDriverWait(driver, 10)
    elements = wait.until(ec.presence_of_all_elements_located((By.XPATH, "//*[contains(text(), 'மேலும் காண்க')]")))
    clickable_elements = []
    for element in elements:
        if element.is_displayed() and element.is_enabled():
            clickable_elements.append(element)

    # Click all the clickable "மேலும் காண்க" elements
    for element in clickable_elements:
        driver.execute_script("arguments[0].click();", element)
    print('clicking complete')
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    # Find all elements with class 'qu-mb--tiny'
    small_classes = soup.find_all(class_='qu-mb--tiny')
    x = 0
    # Loop over the small_classes and find the adjacent 'spacing_log_answer_content' class
    for small_class in small_classes:
        adjacent_class = small_class.find_next(class_='spacing_log_answer_content')
        if adjacent_class:     
            mydict = {}
            question = small_class.text.strip()
            answer = adjacent_class.text.strip()
            if len(question)>30:
                x = x+1
                mydict["question"] = question
                mydict["answer"] = answer
                # Open the JSON file in append mode
                with open(file_name, "a",encoding='utf-8') as f:
                    f.write(str(mydict))
                    f.write('\n')
    print('closing***********************************************')
    driver.quit()    

In [ ]:
from tqdm import tqdm
n=0
for i in tqdm(links):
    n+=1
    try:
        file_name = 'link_{:04d}.txt'.format(n)
        extract_text(i,file_name)
    except:
        pass   

  0%|                                                                                       | 0/143 [00:00<?, ?it/s]

starting******************wait until closing**********
scrolling completed
clicking complete
closing***********************************************


  1%|▌                                                                           | 1/143 [02:24<5:41:47, 144.42s/it]

starting******************wait until closing**********
scrolling completed
clicking complete
closing***********************************************


  1%|█                                                                           | 2/143 [04:53<5:45:58, 147.23s/it]

...

starting******************wait until closing**********
scrolling completed
clicking complete
closing***********************************************


 10%|███████▊                                                                   | 15/143 [35:54<5:08:21, 144.54s/it]

starting******************wait until closing**********


 11%|████████▍                                                                  | 16/143 [36:53<4:11:17, 118.72s/it]

starting******************wait until closing**********


from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
import random
import urllib.parse

url_raw = 'https://ta.quora.com/topic/%E0%AE%85%E0%AE%B1%E0%AE%BF%E0%AE%B5%E0%AE%BF%E0%AE%AF%E0%AE%B2%E0%AF%8D'
url = urllib.parse.unquote(url_raw)
# Load the web page
#options = webdriver.ChromeOptions()
#options.add_argument('--headless')
#driver = webdriver.Chrome(options=options)
driver = webdriver.Chrome()
driver.get(url)
scroll_to_bottom(driver)
print('scrolling completed')
# Find all clickable "மேலும் காண்க" elements
wait = WebDriverWait(driver, 10)
elements = wait.until(ec.presence_of_all_elements_located((By.XPATH, "//*[contains(text(), 'மேலும் காண்க')]")))
clickable_elements = []
for element in elements:
    if element.is_displayed() and element.is_enabled():
        clickable_elements.append(element)

# Click all the clickable "மேலும் காண்க" elements
for element in clickable_elements:
    driver.execute_script("arguments[0].click();", element)
print('clicking complete')

html = driver.page_source

from bs4 import BeautifulSoup
import json
soup = BeautifulSoup(html, 'html.parser')

soup = BeautifulSoup(html, 'html.parser')

# Find all elements with class 'qu-mb--tiny'
small_classes = soup.find_all(class_='qu-mb--tiny')
x = 0
# Loop over the small_classes and find the adjacent 'spacing_log_answer_content' class
for small_class in small_classes:
    adjacent_class = small_class.find_next(class_='spacing_log_answer_content')
    if adjacent_class:     
        mydict = {}
        question = small_class.text.strip()
        answer = adjacent_class.text.strip()
        if len(question)>30:
            x = x+1
            mydict["question"] = question
            mydict["answer"] = answer
            # Open the JSON file in append mode
            with open("qna_quora.txt", "a",encoding='utf-8') as f:
                f.write(str(mydict))          

#finding next sibling
from bs4 import BeautifulSoup
# Create a BeautifulSoup object
soup = BeautifulSoup(html, 'html.parser')
question_elements = soup.find_all(class_='qu-mb--tiny')
# Find all 'qu-mb--tiny' elements and iterate over each of them
count = 0
for question in question_elements:
    # Find the next sibling element with class 'spacing_log_answer_content'
    #answer = question.find_next_sibling(class_='spacing_log_answer_content')  
    # Extract the text from both elements
    if question:
        #question_text = question.text(strip=True)
        #answer_text = answer.text(strip=True)
        # Print the question and answer
        count+=1
        print(count)
        print(question.get_text().strip())
        #print('Answer:', answer_text)
        print('------------------------')
